# Ranking of news search results

In the previous tutorial, you learn how to search news using natural languages. In order to make the search results more useful, you will learn how to rank the search results in this tutorial. We will use the number of view as the score of news articles as it represent the popularity of the articles. The steps include:
1. Create a view count feature group with sample view count dataset
2. Create a feature view that join the news feature group and view count feature group
3. Search news and rank them by view count

## Create a view count feature group

In [ ]:
import pandas as pd

First you create a sample view count dataset of the size of news feature group.

In [ ]:
import random
num_news = 300
df_view = pd.DataFrame({"news_id": list(range(num_news)), "view_cnt": [random.randint(0, 100) for i in range(num_news)]})

In [ ]:
version = 1

Then you create a view count feature group and ingest the data into Hopsworks.

In [ ]:
import hopsworks
proj = hopsworks.login()
fs = proj.get_feature_store()

In [ ]:
view_fg = fs.get_or_create_feature_group(
    name="view_fg",
    primary_key=["news_id"],
    version=version,
    online_enabled=True,
)

view_fg.insert(df_view, write_options={"start_offline_materialization": False})

## Create a feature view 

You need to first get back the news feature group created before for the creation of feature view.

In [ ]:
fg = news_fg = fs.get_or_create_feature_group(
    name="news_fg",
    version=1
)

Now, you create a feature view by joining the news feature group and the view count feature group. Here, you select the heading, and the view count for ranking.

In [ ]:
fv = fs.get_or_create_feature_view(
    "news_view", version=version,
    query=news_fg.select(["heading"]).join(view_fg.select(["view_cnt"]))
)

## Search news and rank 

Same as the previous tutorial, the news description first needs to be encoded by the same LM you used to encoded the news. And then the embedding can be used to search similar news using the feature view.

In [ ]:
news_description = "news about europe"

In [ ]:
!pip install sentence_transformers -q

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import logging
logging.getLogger().setLevel(logging.WARN)

Define some helper functions which sort and print new results.

In [ ]:
def print_news(feature_vectors):
    for feature_vector in feature_vectors:
        print(feature_vector)

In [ ]:
def print_sort_news(feature_vectors):
    # sort the articles by view count
    print("Ranked result:")
    feature_vectors = sorted(feature_vectors, key=lambda x: x[1]*-1)
    print_news(feature_vectors)

Now, you can see the top k results returned by the feature view, which are the headings and the view count. You can also see the ranked results by view count of the top k results.

In [ ]:
feature_vectors = fv.find_neighbors(model.encode(news_description), k=5, feature=news_fg.embedding_heading)
print_news(feature_vectors)
print_sort_news(feature_vectors)

Like the feature group, you can filter results in `find_neighbors` in feature view. You can also use multiple filtering conditions.

In [ ]:
feature_vectors = fv.find_neighbors(model.encode(news_description), k=5, 
                  filter=((news_fg.newstype == "sports") & (news_fg.article.like("europe"))),
                 feature=news_fg.embedding_heading)
print_news(feature_vectors)
print_sort_news(feature_vectors)

You can get back result by providing primary key which is the news id as well.

In [ ]:
feature_vectors = fv.get_feature_vector({"news_id": 10})
print_news([feature_vectors])

## Next step

Now you are able to search articles and rank them by view count. You may be wondering why the view count does not store in the news feature group. You can find the answer and other best practices in the [guide]().